# Data processing and cleaning with Pandas

<!-- ### Basic Processing and Cleaning -->

<!-- ## Marco Forgione -->



A helper class for this lecture:

In [1]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New",Courier,monospace'>{0}</p>{1}
    </div>"""
    def __init__(self,*args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a,eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

Taken from https://github.com/jakevdp/PythonDataScienceHandbook. Useful to display several pandas dataframes in one line... 

### Pandas recap

The pandas module provides: 

* a 2D labeled data structure: ``pd.DataFrame``
* a 1D labeled data structure: ``pd.Series``

We typically read a dataframe from an external source (e.g. CSV file) and perform manipulations on it. <!-- ## Here,let us work on artificial data. -->

In [2]:
# usual imports...
import pandas as pd
import numpy as np

In [3]:
# create artificial data (normally we read from a source...)
data = pd.DataFrame(np.random.randn(3, 2), columns=["A", "B"], index=["a", "b", "c"]) #  = pd.read_csv("filename.csv")
data # this is a pandas DataFrame

,A,B
a,-0.315249,1.232340
b,0.669884,0.328862
c,-1.853765,1.003830


In [4]:
data["B"] # this is a Series

a    1.232340
b    0.328862
c    1.003830
Name: B, dtype: float64

In [5]:
data.loc["c"] # this is also a Series...

A   -1.853765
B    1.003830
Name: c, dtype: float64

We have seen how to select rows/columns and filter according to logical conditions (find city in Switzerland...) in Lecture 3. <br/>
Today, we do some basic processing!

### Unary Operations

**Unary**  operations from ``numpy`` can be applied **element-wise** to all the elements of a series or dataframe. 

In [6]:
data # a dataframe with random data, from the previous slide

,A,B
a,-0.315249,1.232340
b,0.669884,0.328862
c,-1.853765,1.003830


In [7]:
np.sin(data["A"]) # np.sin function applied to the series data["A"]

a   -0.310053
b    0.620895
c   -0.960231
Name: A, dtype: float64

In [8]:
data_sin = np.sin(data) # np.sin also works on the dataframe...
data_exp = np.exp(data) # np.sin,np.cos,np.exp,np.log
data_half_sq = data**2/2.0 # broadcasting of scalar value 2.0 to all elements of the dataframe also works as expected
display('data','data_sin','data_exp','data_half_sq') # the display class that we defined in slide 2...

,A,B
a,-0.315249,1.232340
b,0.669884,0.328862
c,-1.853765,1.003830
,A,B
a,-0.310053,0.943268
b,0.620895,0.322966
c,-0.960231,0.843534
,A,B
a,0.729607,3.429245
b,1.954012,1.389386


### Unary Operations

Generic unary operations may be applied element-wise via the **map** (for ``pd.Series``) and **applymap** (for ``pd.DataFrame``) methods:

In [9]:
data = pd.DataFrame(np.random.randint(0, 10, size=(3, 2)), columns = ["A", "B"])
data

,A,B
0,9,9
1,5,7
2,7,7


In [10]:
# The element-wise operation that we want to apply to our data
def squared_plus1(x): 
    return x**2 + 1

In [11]:
data["A"].apply(squared_plus1) # apply squared_plus1 to all elements of the series. Note: data["A"] is a pd.Series!
# data["A"]**2 + 1

0    82
1    26
2    50
Name: A, dtype: int64

In [12]:
data_1 = data.applymap(squared_plus1) # apply squared_plus1 to all elements of the df
display('data', 'data_1')

,A,B
0,9,9
1,5,7
2,7,7
,A,B
0,82,82
1,26,50
2,50,50


Anonymous *lambda* functions may also be used:

In [13]:
data["A"].apply(lambda x: x**2 + 1); # same as data["A"].apply(squared_plus1)

### Binary operations

Things get more interesting for binary operations. Pandas automatically **aligns data according to the index** of a Series:

In [14]:
area = pd.Series({'Alaska': 1_723_337,'Texas': 695_662,'California': 423_967}, name='area')
population = pd.Series({'California': 38_332_521,'Texas': 26_448_193,'New York': 19_651_127}, name='population')

In [15]:
area

Alaska        1723337
Texas          695662
California     423967
Name: area, dtype: int64

In [16]:
population

California    38332521
Texas         26448193
New York      19651127
Name: population, dtype: int64

In [17]:
density = population/area

In [18]:
density

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

* The result includes the indices of the two series (Alaska, California, New York, Texas)
* For the common indices (California, Texas), the operation is actually performed
* For the indices that are not in both series (Alaska, New York), the result is ``np.nan``

### Binary Operations

A similar alignment mechanism applies to dataframes. Data is aligned according to both row and column indices

In [19]:
df1 = pd.DataFrame(np.random.randint(0, 10, size=(4,3)), columns=["A","B","C"], index=['a','b','c','d'])
df2 = pd.DataFrame(np.random.randint(0, 10, size=(3,3)), columns=["B","C","D"], index=['a','b','c'])
df_sum = df1 + df2
display('df1','df2','df_sum')

df1
   A  B  C
a  4  6  7
b  8  5  1
c  3  3  3
d  3  4  3

df2
   B  C  D
a  4  3  9
b  5  9  7
c  7  0  6

df_sum
    A     B     C   D
a NaN  10.0  10.0 NaN
b NaN  10.0  10.0 NaN
c NaN  10.0   3.0 NaN
d NaN   NaN   NaN NaN

* The result ``df_sum=df1+df2`` includes rows and column indices from both ``df1`` and ``df2``
* Columns A and D in ``df_sum`` are ``np.nan``
* Row ``d`` in ``df_sum`` is np.nan

The binary operation is applied to all the elements where it makes sense to do it.

### Add a new column

We can add a new column to an existing dataframe based using the dictionary-like syntax: ``data["key"] = 1D array_like``. <br/>
The 1D array_like may be a seried obtained with unary/binary operations on existing columns.

In [20]:
data = pd.DataFrame(np.random.randn(2, 2), columns=["X", "Y"])
data

,X,Y
0,2.136322,-0.618581
1,-0.106840,1.276946


In [21]:
data["R"] = data["X"]**2 + data["Y"]**2 # assign data["R"] to a pd.Series using dictionaty-like syntax
data

,X,Y,R
0,2.136322,-0.618581,4.946515
1,-0.106840,1.276946,1.642007


In [22]:
data["A"] = ["a", "b"] # assign a list to a new column
data["B"] = np.arange(2) # assign a numpy array to a new column
data

,X,Y,R,A,B
0,2.136322,-0.618581,4.946515,a,0
1,-0.106840,1.276946,1.642007,b,1


We can speficy the position for the new column using the ``insert`` method of the dataframe:

In [23]:
data.insert(3, "W", data["X"]/data["R"] ) # insert in position 2 (third column),with name W
data

,X,Y,R,W,A,B
0,2.136322,-0.618581,4.946515,0.431884,a,0
1,-0.106840,1.276946,1.642007,-0.065067,b,1


### Aggregation methods for series

* Statistical operations such as ``mean``, ``min``, ``max``, ``std``, are built-in methods of ``pd.Series``.

In [24]:
population = pd.Series({'Bellinzona': 17_744,'Lugano': 62_615,'Mendrisio': 11_554,'Stabio': 4_510,'Lausanne': 140_000,'Bern': 133_115}, name='population')

In [25]:
population.mean(),population.min(),population.max(),population.std()

(61589.666666666664, 4510, 140000, 61561.68250029126)

The methods above are **aggregations**: they transform a sequence (``pd.Series``) to a scalar (``float`` or ``int``). <br/>


* The series methods ``sum`` and ``count`` are other common and useful aggregations. 

In [26]:
population.count(), population.sum() # number of cities,sum of the population of the cities

(6, 369538)

### Aggregation methods for dataframes
The aggregations ``mean``, ``min``, ``max``, ``std``, ``count``, ``sum`` are also built-in for dataframes. They are applied either row-wise or column-wise

In [27]:
df = pd.DataFrame(np.random.randint(0, 10, size=(3, 2)), columns = ["A", "B"], index=['a', 'b', 'c'])
df

,A,B
a,9,2
b,8,5
c,9,9


By default, the operations are applied for each column, to the elements in the rows

In [28]:
col_sum = df.sum(axis="index") # df.sum(axis="index") by default -> compute sum for each column
col_sum

A    26
B    16
dtype: int64

Thus, ``col_sum["A"]`` is the sum of the elements of column A, computed over rows a, b, c.

By speficying the option ``axis=columns``, the operations are computed for each row, to the elements in the columns:

In [29]:
row_sum = df.sum(axis="columns")
row_sum

a    11
b    13
c    18
dtype: int64

Thus, ``row_sum["a"]`` is the mean of row a, computed over columns A,B,C.

### The describe method

The ``describe`` dataframe method summarizes several **column statistics** and returns a dataframe:

In [30]:
data = pd.DataFrame(np.random.randn(4, 2), columns=["X", "Y"])
data["Z"] = ["a", "b", "c", "d"]
data

,X,Y,Z
0,-0.273994,1.327902,a
1,0.954017,1.055717,b
2,-0.617833,1.416726,c
3,-0.558905,0.038506,d


In [31]:
data.describe() # dataframe summary statistics

,X,Y
count,4.000000,4.000000
mean,-0.124179,0.959712
std,0.734310,0.633052
min,-0.617833,0.038506
25%,-0.573637,0.801414
50%,-0.416450,1.191809
75%,0.033009,1.350108
max,0.954017,1.416726


Describe is also available for a series

In [32]:
data['Z'].describe() # series summary statistics

count     4
unique    4
top       c
freq      1
Name: Z, dtype: object

NOTE: for a dataframe, ``describe`` works for numeric columns only. Non-numeric columns are automatically skipped...

### Custom row- and column-wise dataframe operations


Custom row-wise or column-wise operations via the dataframe ``apply`` method. 
It may be used to compute custom statistics.

In [33]:
df = pd.DataFrame(np.random.rand(3, 2), columns=["A", "B"], index=['a', 'b', 'c'])
df

,A,B
a,0.182816,0.386462
b,0.105232,0.225686
c,0.901712,0.755728


In [34]:
def numerical_range(x):
    return x.max() - x.min()

In [35]:
df.apply(numerical_range, axis="index") # numerical range, by columns
# numerical_range is an aggregation function. Thus, when applied to a dataframe, it will return a Series...

A    0.796480
B    0.530041
dtype: float64

In [36]:
df.apply(numerical_range, axis="columns") # numerical range,by rows

a    0.203647
b    0.120454
c    0.145985
dtype: float64

Note: the function ``numerical_range`` expects a ``pd.Series`` corresponding to a column (or a row) of the dataframe. 


Anonymous *lambda* functions can also be used:

In [37]:
df.apply(lambda x: x["a"] + x["b"] - x["c"]) 

A   -0.613665
B   -0.143579
dtype: float64

In [38]:
df.apply(lambda x: np.sqrt(x["A"]**2 + x["B"]**2),axis="columns") # can also assign it to new column: df["C"] = df.apply(lambda row: np.sqrt(row["A"]**2 + row["B"]**2),axis="columns")

a    0.427522
b    0.249014
c    1.176524
dtype: float64

### Sorting

Dataframes may be sorted by column values...

In [39]:
df = pd.DataFrame(np.random.randint(0, 10, size=(4, 2)), columns=["A", "B"], index=['a', 'd', 'c', 'b'])
df_sort_A_asc = df.sort_values(by=["A"]) # default: ascending order (from small to big)
df_sort_A_dsc = df.sort_values(by=["A"], ascending=False) # (from big to small)
display('df', 'df_sort_A_asc', 'df_sort_A_dsc')

,A,B
a,8,0
d,4,2
c,9,8
b,8,0
,A,B
d,4,2
a,8,0
b,8,0
c,9,8
,A,B


...or by index value

In [40]:
df1.sort_index() # sort dataframe according to its index (alphabetical order)

,A,B,C
a,4,6,7
b,8,5,1
c,3,3,3
d,3,4,3


### Inplace operations

Sorting may be performed inplace setting the ``inplace`` option to True. Then:
 * The original dataframe **is modified**
 * The return value is ``None``

In [41]:
df = pd.DataFrame(np.random.randint(0, 10, size=(6, 2)), columns=["A", "B"])
df

,A,B
0,4,5
1,3,6
2,5,4
3,7,8
4,1,6
5,4,0


In [42]:
ret_val = df.sort_values(by=["A"], inplace=True)
df

,A,B
4,1,6
1,3,6
0,4,5
5,4,0
2,5,4
3,7,8


In [43]:
ret_val is None

True

* Several numpy/pandas methods (typically data transformations) have an ``inplace`` option. Look up in the documentation!
* On large datasets, inplace operations may be faster/save memory. 

### Counting

Common tasks: check distinct values of a series, count their occurrences.

In [44]:
ser = pd.Series(np.array([0, 0, 0, 1, 1, 0, 0, 1, 0, 0]), name="bits")
ser

0    0
1    0
2    0
3    1
4    1
5    0
6    0
7    1
8    0
9    0
Name: bits, dtype: int64

In [45]:
ser.count() # the series contains 10 elements

10

In [46]:
ser.unique() # the series contains the distinct values 0 and 1

array([0, 1])

In [47]:
ser.value_counts() # the value 0 occurs 7 times,the value 1 occurs 3 times

0    7
1    3
Name: bits, dtype: int64

Useful to understand whether a variable is numerical (in principle, infinite number of possible values) or categorical (finite possible values).

### String manipulation

Series of string objects have a ``str`` attribute containing useful methods for string manipulation.

In [48]:
df = pd.DataFrame(np.random.randint(1, 10, size=(3, 4)), columns=["Area", "Population", "Latitude", "Longitude"])
df.insert(0, "City", ["Lugano", "Geneva", "Zurich"]) # insert column at position 0
df

,City,Area,Population,Latitude,Longitude
0,Lugano,9,7,8,1
1,Geneva,8,7,1,7
2,Zurich,2,9,8,9


In [49]:
df['City'].str

Example: convert to uppercase/ to lowecase

In [50]:
df['City'] = df['City'].str.upper() # to uppercase
df

,City,Area,Population,Latitude,Longitude
0,LUGANO,9,7,8,1
1,GENEVA,8,7,1,7
2,ZURICH,2,9,8,9


In [51]:
df['City'] = df['City'].str.lower() # to lowercase
df

,City,Area,Population,Latitude,Longitude
0,lugano,9,7,8,1
1,geneva,8,7,1,7
2,zurich,2,9,8,9


Many more methods available. Look up in the documentation!

### Date manipulation

Dates may be simply represented as strings:

In [52]:
df = pd.DataFrame({"date": ["01/12/2020", "02/01/2021", "03/02/2021", "04/03/2021"], "val": np.random.randn(4)})
df

,date,val
0,01/12/2020,0.880854
1,02/01/2021,-1.243332
2,03/02/2021,1.126850
3,04/03/2021,-0.120213


However, it is convenient to represent dates with a specific data type. Pandas can parse several date formats. A format string may be given as argument.

In [53]:
df["date"] = pd.to_datetime(df["date"], dayfirst=True)  # specify that the first field is the day
#df["date"] = pd.to_datetime(df["date"],format='%d/%m%/Y')  # specity a format
df["date"]

0   2020-12-01
1   2021-01-02
2   2021-02-03
3   2021-03-04
Name: date, dtype: datetime64[ns]

A datetime series has a ``dt`` attribute with useful tools for datetime handling:

In [54]:
df["date"].dt

In [55]:
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day
df

,date,val,year,month,day
0,2020-12-01,0.880854,2020,12,1
1,2021-01-02,-1.243332,2021,1,2
2,2021-02-03,1.126850,2021,2,3
3,2021-03-04,-0.120213,2021,3,4


# Cleaning


Real data are often messy/unconsistend. 

A good portion of the data scientits' work is to clean and prepare data!

### Missing data

Missing values are generally represented with ``np.nan`` or None

In [56]:
df = pd.DataFrame(np.random.randn(6,5), columns=["A","B","C","D","E"])
df.iloc[0, 0] = np.nan
df.iloc[1, 1] = np.nan
df.iloc[2, 3] = np.nan
df.iloc[3, 3] = np.nan
df

,A,B,C,D,E
0,NaN,0.112926,-0.705262,-0.976229,-0.929942
1,0.496511,NaN,-1.637904,1.063799,0.295775
2,0.210966,-0.815134,0.578499,NaN,-0.784512
3,0.201139,-1.561025,-0.818954,NaN,0.594385
4,-0.501074,0.628234,0.332443,1.518045,-0.800002
5,0.378546,0.225944,-0.092519,-0.219541,-0.279926


We can identify missing values using the ``isna`` method:

In [57]:
df.isna() # or df.isnull()

,A,B,C,D,E
0,True,False,False,False,False
1,False,True,False,False,False
2,False,False,False,True,False
3,False,False,False,True,False
4,False,False,False,False,False
5,False,False,False,False,False


The ``isna`` operation is often combined with the ``any`` aggregation:

In [58]:
df.isna().any() # columns where at least one element is missing
# df.isna().any(axis="columns") # rows where at least one element is missing

A     True
B     True
C    False
D     True
E    False
dtype: bool

### Missing data: counting columns/rows with missing values

As in the previous slide: let us look for columns with missing values

In [59]:
df.isna().any() # all columns with missing values

A     True
B     True
C    False
D     True
E    False
dtype: bool

Then, 3 columns (A, B, D) contain missing values. Two columns (C, E) have no missing.


If we had, say, 100 columns, if would be tedious to count them by hand! Solution: use ``value_counts``!

In [60]:
df.isna().any().value_counts() # count columns with missing/no missing values

True     3
False    2
dtype: int64

We can use the same trick for rows:

In [61]:
df.isna().any(axis="columns")

0     True
1     True
2     True
3     True
4    False
5    False
dtype: bool

In [62]:
df.isna().any(axis="columns").value_counts()

True     4
False    2
dtype: int64

Then, 4 rows contain missing values, 2 rows have no missing.

### Missing data handling

Several data analysis/machine learning models do not work with missing values. We may have to get rid of them!

* Sometimes the "right" thing to do is to drop incomplete rows:

In [63]:
df.dropna() # only keeps complete rows, equivalent to df.dropna(axis="index")

,A,B,C,D,E
4,-0.501074,0.628234,0.332443,1.518045,-0.800002
5,0.378546,0.225944,-0.092519,-0.219541,-0.279926


This makes sense when there are just a few rows with missing values

* Sometimes the "right" thing to do is to drop incomplete columns:

In [64]:
df.dropna(axis="columns") # only keeps complete columns

,C,E
0,-0.705262,-0.929942
1,-1.637904,0.295775
2,0.578499,-0.784512
3,-0.818954,0.594385
4,0.332443,-0.800002
5,-0.092519,-0.279926


* Sometimes we may want to replace ``np.nan`` with a numeric value:

In [65]:
df.fillna(0.0) # replace all np.nans with 0.0

,A,B,C,D,E
0,0.000000,0.112926,-0.705262,-0.976229,-0.929942
1,0.496511,0.000000,-1.637904,1.063799,0.295775
2,0.210966,-0.815134,0.578499,0.000000,-0.784512
3,0.201139,-1.561025,-0.818954,0.000000,0.594385
4,-0.501074,0.628234,0.332443,1.518045,-0.800002
5,0.378546,0.225944,-0.092519,-0.219541,-0.279926


This makes sense if 0.0 is a reasonable default value for missing information. Not always the case!

### Missing data handling cont'd
Tailored solutions for each case may be required.

In [66]:
df

,A,B,C,D,E
0,NaN,0.112926,-0.705262,-0.976229,-0.929942
1,0.496511,NaN,-1.637904,1.063799,0.295775
2,0.210966,-0.815134,0.578499,NaN,-0.784512
3,0.201139,-1.561025,-0.818954,NaN,0.594385
4,-0.501074,0.628234,0.332443,1.518045,-0.800002
5,0.378546,0.225944,-0.092519,-0.219541,-0.279926


We may want to replace missing values with the column mean, with the column median, or with some kind of interpolation technique.

In [67]:
df['A'] = df['A'].fillna(df['A'].mean()) # for column A, replace missing values with the column mean
df['B'] = df['B'].fillna(df['B'].median()) # for column B, replace missing values with the column median
df['D'] = df['D'].fillna(method='ffill') # for column D, forward-fill with previous valid point

In [68]:
df

,A,B,C,D,E
0,0.157218,0.112926,-0.705262,-0.976229,-0.929942
1,0.496511,0.112926,-1.637904,1.063799,0.295775
2,0.210966,-0.815134,0.578499,1.063799,-0.784512
3,0.201139,-1.561025,-0.818954,1.063799,0.594385
4,-0.501074,0.628234,0.332443,1.518045,-0.800002
5,0.378546,0.225944,-0.092519,-0.219541,-0.279926


There is no general rule for handling missing data. For instance, forward-fill interpolation makes sense for timeseries data.

### Replacing Values

Sometimes a certain value, e.g. ``-1`` is used to code missing info in the dataset. The ``replace`` method of ``pd.Series`` comes in handy.

In [69]:
ser = pd.Series(np.random.rand(10))
ser[4] = -1
ser[8] = -1
ser

0    0.413577
1    0.230166
2    0.567934
3    0.820276
4   -1.000000
5    0.255347
6    0.036790
7    0.906758
8   -1.000000
9    0.790447
dtype: float64

In [70]:
ser.replace(-1, np.nan)

0    0.413577
1    0.230166
2    0.567934
3    0.820276
4         NaN
5    0.255347
6    0.036790
7    0.906758
8         NaN
9    0.790447
dtype: float64

### Dropping rows/columns

It is common to drop one or more columns from a dataframe:

In [71]:
df = pd.DataFrame(np.random.randint(1,10,size=(3,4)),columns=["Area","Population","Latitude","Longitude"],index=["Lugano","Geneva","Zurich"])
df

,Area,Population,Latitude,Longitude
Lugano,1,5,2,6
Geneva,3,2,6,7
Zurich,7,7,3,1


In [72]:
df.drop(["Population"], axis="columns") # drop also has the inplace option. 

,Area,Latitude,Longitude
Lugano,1,2,6
Geneva,3,6,7
Zurich,7,3,1


In [73]:
df.drop(["Population","Area"], axis="columns")

,Latitude,Longitude
Lugano,2,6
Geneva,6,7
Zurich,3,1


With a similar sintax, we can drop rows:

In [74]:
df.drop("Zurich") # or df.drop("Zurich",axis="rows") 

,Area,Population,Latitude,Longitude
Lugano,1,5,2,6
Geneva,3,2,6,7


### Renaming rows/columns

Fix some typos in rows and columns names. 

We can use the ``rename`` method of ``pd.DataFrame``:

In [75]:
df = pd.DataFrame(np.random.randint(1,10,size=(3,4)), columns=["Area","Popluation","Latitude","Longitude"], index=["Lugano","Geneva","Zuric"])
df

,Area,Popluation,Latitude,Longitude
Lugano,7,3,6,5
Geneva,6,3,4,9
Zuric,8,3,5,2


In [76]:
df = df.rename(columns={"Popluation": "Population"}) # rename some column names
df = df.rename(index={"Zuric": "Zurich"}) # rename some index names
df

,Area,Population,Latitude,Longitude
Lugano,7,3,6,5
Geneva,6,3,4,9
Zurich,8,3,5,2


In [77]:
df.index.name = "City" # the index itself may have a name...

In [78]:
df

,Area,Population,Latitude,Longitude
City,,,,
Lugano,7,3,6,5
Geneva,6,3,4,9
Zurich,8,3,5,2


### Renaming rows/columns

Alternatively, we can assign the row and column index of the dataframe: 

In [79]:
df = pd.DataFrame(np.random.randint(1, 10, size=(3 , 4)),
                  columns=["Area","Popluation","Latitude","Longitude"],
                  index=["Lugano","Geneva","Zuric"])
df

,Area,Popluation,Latitude,Longitude
Lugano,1,7,2,8
Geneva,1,9,5,2
Zuric,5,7,1,5


In [80]:
df.columns

Index(['Area', 'Popluation', 'Latitude', 'Longitude'], dtype='object')

In [81]:
df.columns = ['Area','Population','Latitude','Longitude']

In [82]:
df.index = ["Lugano","Geneva","Zurich"]

In [83]:
df

,Area,Population,Latitude,Longitude
Lugano,1,7,2,8
Geneva,1,9,5,2
Zurich,5,7,1,5


### Dropping duplicates

Duplicate rows may be suspicious. We can easily identify and remove them.

In [84]:
df = pd.DataFrame(np.array([[0,0], [0,1], [1,0], [1,1], [0,0]]), columns=["A", "B"])
df

,A,B
0,0,0
1,0,1
2,1,0
3,1,1
4,0,0


In [85]:
df.duplicated()

0    False
1    False
2    False
3    False
4     True
dtype: bool

In [86]:
df.drop_duplicates()

,A,B
0,0,0
1,0,1
2,1,0
3,1,1
